In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

%run 1.tokenizer.ipynb

tokenizer

In [2]:
%run 2.dataset.ipynb

dataset = Dataset(negative_label=False, with_answer=False)

len(dataset), dataset[0]

(2000000, {'text': '1577-8826=', 'label': 1})

In [3]:
from transformers import AutoModelForCausalLM

model_actor = AutoModelForCausalLM.from_pretrained('model/ppo').to(device)

model_actor.config

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


GPTNeoXConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "model/ppo",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_bias": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 1,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "partial_rotary_factor": 0.25,
  "rope_scaling": null,
  "rope_theta": 10000,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.48.0",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 19
}

In [4]:
#====question====
question = random.choices(dataset, k=12)
question = [i['text'] for i in question]

question = tokenizer(question,
                     device=device,
                     add_bos_token=True,
                     add_eos_token=False,
                     padding_side='left')['input_ids']

#====gen====
gen = model_actor.generate(input_ids=question,
                           min_length=-1,
                           max_length=50,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           top_k=0.0,
                           top_p=1.0,
                           do_sample=True)
gen = gen[:, question.shape[1]:]

for q, g in zip(question, gen):
    q = tokenizer.decode(q)
    g = tokenizer.decode(g)
    a = q
    a = a[a.index(tokenizer.bos_token) + 1:]
    a = a[:a.index('=')]
    a = int(eval(a))

    print({'question': q, 'answer': a, 'gen': g})


{'question': 'B278-4290=', 'answer': -4012, 'gen': '-4012E'}
{'question': 'B1512*3394=', 'answer': 5131728, 'gen': '4956128E'}
{'question': 'B5517*6921=', 'answer': 38183157, 'gen': '37428057E'}
{'question': 'B9892*8802=', 'answer': 87069384, 'gen': '87248984E'}
{'question': 'B2157-6686=', 'answer': -4529, 'gen': '-4529E'}
{'question': 'B9746-4099=', 'answer': 5647, 'gen': '4647E'}
{'question': 'B2008-3150=', 'answer': -1142, 'gen': '-1142E'}
{'question': 'B5949-9098=', 'answer': -3149, 'gen': '-3149E'}
{'question': 'B7138/6154=', 'answer': 1, 'gen': '1E'}
{'question': 'B1306*2185=', 'answer': 2853610, 'gen': '3195810E'}
{'question': 'B1644*103=', 'answer': 169332, 'gen': '218012E'}
{'question': 'B3332-9696=', 'answer': -6364, 'gen': '-6364E'}
